In [ ]:
!pip install xgboost

In [ ]:
!pip install --upgrade gspread

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.1.4
    Uninstalling gspread-6.1.4:
      Successfully uninstalled gspread-6.1.4


In [ ]:
!pip install gspread_formatting

In [ ]:
!pip install nltk

In [ ]:
from google.colab import auth
auth.authenticate_user()


import calendar
from datetime import datetime
import pandas as pd
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
# Libros
libro = gc.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original

In [ ]:
# SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
# TablaCausas=libro.get_worksheet(9)
# TablaEfectos=libro.get_worksheet(8)
# Actividades=libro.get_worksheet(5)
# CoinversoresTabla=libro.get_worksheet(11)
# AliadosDelProyecto=libro.get_worksheet(2)

In [ ]:
TablaFinImpacto=libro.get_worksheet(7)
TablaFinImpacto_d = TablaFinImpacto.get_all_values()
TablaFinImpacto_df=pd.DataFrame(TablaFinImpacto_d)
TablaFinImpacto_df=TablaFinImpacto_df.iloc[2:,[0,4]]
TablaFinImpacto_df.columns=["ID", "Respuesta"]
TablaFinImpacto_df

,ID,Respuesta
2,3694099,Fin o Impacto: Atención directa e integral a p...
3,3693655,Fin o Impacto: Atención directa e integral a p...
4,3693455,Fin o Impacto: Intervención directa en viviend...
5,3692684,Fin o Impacto: Distribución directa de aliment...
6,3692415,Fin o Impacto: Acompañamiento formativo a pers...
...,...,...
370,3667535,"Fin Componente, a continuación se enlistan los..."
371,3667535,"Fin Componente, a continuación se enlistan los..."
372,3667029,"Fin Componente, a continuación se enlistan los..."
373,3667016,"Fin Componente, a continuación se enlistan los..."


In [ ]:
Cálculo_Puntaje=gc.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[0,1]]
Rubrica_df.columns=["Mision", "Clasificación"]
Rubrica_df["Mision"]=Rubrica_df["Mision"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
Rubrica_df

,Mision,Clasificación
1,Unicamente ofrecer un producto para satisfacer...,0
2,Busca atender una necesidad de los/as titulare...,1
3,Busca generar cambios a corto plazo en los/as ...,2
4,Busca generar cambios a mediano plazo en los t...,3
5,Busca generar cambios a mediano plazo en titul...,4
6,Busca generar cambios a largo plazo en titular...,5
7,Busca generar cambios a largo plazo en titular...,6


In [ ]:
Puntajes=gc.open_by_key("1cXhgAQDmUhOlgHqjkLghYbPtqY7MqE66H2g5VUt_Mhs")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,1]]
Puntaje_df.columns=["ID","Mision"]
Puntaje_df

,ID,Mision
2,3690465,Busca generar cambios a corto plazo en los/as ...
3,3689775,Busca atender una necesidad de los/as titulare...
4,3685358,Busca generar cambios a largo plazo en titular...
5,3686853,Busca generar cambios a corto plazo en los/as ...
6,3671758,Busca generar cambios a mediano plazo en los t...
...,...,...
270,3691475,Busca generar cambios a mediano plazo en los t...
271,3673132,Busca generar cambios a largo plazo en titular...
272,3673795,Busca generar cambios a mediano plazo en los t...
273,3686156,Busca generar cambios a mediano plazo en titul...


In [ ]:
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='Mision', how='inner')
df_final= pd.merge(df_sfinal,TablaFinImpacto_df, on="ID", how="inner")
df_final=df_final.iloc[:,[3,2]]
df_final['Respuesta'] = df_final['Respuesta'].apply(quitar_acentos)
df_final['Respuesta'] = df_final['Respuesta'].astype(str)
df_final['Clasificación'] = df_final['Clasificación'].apply(lambda x: x - 2)
df_final

,Respuesta,Clasificación
0,Fin o Impacto: Desarrollo de habilidades en pr...,0
1,"Fin Componente, a continuacion se enlistan los...",4
2,Fin o Impacto: Atencion directa con paquetes d...,0
3,"Fin Componente, a continuacion se enlistan los...",1
4,"Fin Componente, a continuacion se enlistan los...",1
...,...,...
277,"Fin Componente, a continuacion se enlistan los...",3
278,Fin o Impacto: Atencion en servicios de salud ...,1
279,Fin o Impacto: Atencion en servicios de salud ...,1
280,"Fin Componente, a continuacion se enlistan los...",2


In [ ]:
import nltk

nltk.download('all')
text = list(df_final['Respuesta'])
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

In [ ]:
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)




#assign corpus to data['text']

df_final['Respuesta'] = corpus

# data.head()

In [ ]:
# Create Feature and Label sets

X = df_final['Respuesta']

y = df_final['Clasificación']




# train test split (66% train - 33% test)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

#ANN

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=1000)  # 'num_words' define el número máximo de palabras que queremos considerar
tokenizer.fit_on_texts(X_train)  # Ajustamos el tokenizer al conjunto de entrenamiento

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Rellenar las secuencias para que todas tengan la misma longitud
maxlen = 100  # Puedes ajustar este valor dependiendo de la longitud promedio de tus textos
X_train_pad = pad_sequences(X_train_seq, padding='post', maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, padding='post', maxlen=maxlen)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense


model = Sequential()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam

# Crear el modelo LSTM
model = Sequential()

model.add(Embedding(input_dim=5000, output_dim=128, input_length=X_train_pad.shape[1]))

model.add(LSTM(128, return_sequences=True))  # 'return_sequences=True' para devolver la salida de cada celda LSTM
model.add(Dropout(0.5))  # Capa Dropout para prevenir overfitting

model.add(LSTM(128, return_sequences=True))  # Devolvemos secuencias para las siguientes capas LSTM
model.add(Dropout(0.5))

model.add(LSTM(64, return_sequences=False))  # Última capa LSTM, no necesitamos devolver secuencias
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))  # Esta capa es opcional, pero puede mejorar el rendimiento


num_classes = 5  # Para clasificación multiclase
model.add(Dense(num_classes, activation='softmax'))  # Usamos 'softmax' para clasificación multiclase

model.compile(loss='sparse_categorical_crossentropy',  # Usa 'binary_crossentropy' si es clasificación binaria
              optimizer=Adam(),
              metrics=['accuracy'])

# Ver el resumen del modelo
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train_pad, y_train, epochs=100, batch_size=32, validation_data=(X_test_pad, y_test))

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 107ms/step - accuracy: 0.2948 - loss: 1.5819 - val_accuracy: 0.3684 - val_loss: 1.4593
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.3561 - loss: 1.5363 - val_accuracy: 0.3684 - val_loss: 1.4480
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.2685 - loss: 1.5574 - val_accuracy: 0.3684 - val_loss: 1.4498
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3038 - loss: 1.5553 - val_accuracy: 0.3684 - val_loss: 1.4648
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3106 - loss: 1.5804 - val_accuracy: 0.3684 - val_loss: 1.4565
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2388 - loss: 1.5127 - val_accuracy: 0.3684 - val_loss: 1.4697
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.2819 - loss: 1.5304 - val_accuracy: 0.3684 - val_loss: 1.4726
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3285 - loss: 1.4814 - val_accuracy: 0.3684 - val_loss

#Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Convert text to numerical vectors using TF-IDF
    ('nb', MultinomialNB())  # Use KNN classifier to classify the data
])

parameters = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'nb__alpha': [0.1, 0.5, 1, 2, 5]
}

grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

best_pipeline = grid_search.best_estimator_
predictions_MNB = best_pipeline.predict(X_test)

# imprimimos el reporte de la clasificación
# print(classification_report(y_test, predictions_MNB))

Fitting 5 folds for each of 30 candidates, totalling 150 fits


In [ ]:
# # Importación de la librería necesaria
# from sklearn.naive_bayes import MultinomialNB

# # Crear el modelo Naive Bayes
# nb = MultinomialNB()

# # Entrenar el modelo utilizando los datos de entrenamiento
# nb.fit(X_train_cv, y_train)


# # Generar las predicciones
# predictions_MNB = nb.predict(X_test_cv)

# # Mostrar las predicciones
# predictions_MNB

array([1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 0, 1,
       1, 1, 1, 0, 1, 1, 2, 1, 1, 0, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 0,
       1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 2, 1])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, roc_curve, auc, log_loss, matthews_corrcoef

accuracy_MNB = accuracy_score(y_test, predictions_MNB)

conf_matrix_MNB = confusion_matrix(y_test, predictions_MNB)

precision_MNB = precision_score(y_test, predictions_MNB, average='weighted')  # 'weighted' es útil en clasificación multiclase

recall_MNB = recall_score(y_test, predictions_MNB, average='weighted')  # 'weighted' es útil en clasificación multiclase

f1_MNB = f1_score(y_test, predictions_MNB, average='weighted')  # 'weighted' es útil en clasificación multiclase

report = classification_report(y_test, predictions_MNB)

print(f'Accuracy: {accuracy_MNB:.4f}')
print('Confusion Matrix:')
print(conf_matrix_MNB)
print(f'Precision (weighted): {precision_MNB:.4f}')
print(f'Recall (weighted): {recall_MNB:.4f}')
print(f'F1-Score (weighted): {f1_MNB:.4f}')
print('Classification Report:')
print(report)


Accuracy: 0.4035
Confusion Matrix:
[[ 3 11  0  0  0]
 [ 1 20  0  0  0]
 [ 0 10  0  0  0]
 [ 1  9  0  0  0]
 [ 0  2  0  0  0]]
Precision (weighted): 0.2891
Recall (weighted): 0.4035
F1-Score (weighted): 0.2794
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.21      0.32        14
           1       0.38      0.95      0.55        21
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00        10
           4       0.00      0.00      0.00         2

    accuracy                           0.40        57
   macro avg       0.20      0.23      0.17        57
weighted avg       0.29      0.40      0.28        57



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
import pickle
filename = "NBbueno.pkl"
outfile = open(filename, 'wb')

pickle.dump(best_pipeline, outfile, 2)
outfile.close()

## XGB boost

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)


X_train_cv.shape

(225, 237)

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

model.fit(X_train_cv, y_train)

predictions_XGB = model.predict(X_test_cv)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:26:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
accuracy_XGB = accuracy_score(y_test, predictions_XGB)

conf_matrix_XGB = confusion_matrix(y_test, predictions_XGB)

precision_XGB = precision_score(y_test, predictions_XGB, average='weighted')  # 'weighted' es útil en clasificación multiclase

recall_XGB = recall_score(y_test, predictions_XGB, average='weighted')  # 'weighted' es útil en clasificación multiclase

f1_XGB = f1_score(y_test, predictions_XGB, average='weighted')  # 'weighted' es útil en clasificación multiclase

report_XGB = classification_report(y_test, predictions_XGB)

print(f'Accuracy: {accuracy_XGB:.4f}')
print('Confusion Matrix:')
print(conf_matrix_XGB)
print(f'Precision (weighted): {precision_XGB:.4f}')
print(f'Recall (weighted): {recall_XGB:.4f}')
print(f'F1-Score (weighted): {f1_XGB:.4f}')
print('Classification Report:')
print(report_XGB)


Accuracy: 0.3860
Confusion Matrix:
[[ 0 10  4  0  0]
 [ 0 19  2  0  0]
 [ 0  7  3  0  0]
 [ 0  7  3  0  0]
 [ 0  0  2  0  0]]
Precision (weighted): 0.2004
Recall (weighted): 0.3860
F1-Score (weighted): 0.2626
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.44      0.90      0.59        21
           2       0.21      0.30      0.25        10
           3       0.00      0.00      0.00        10
           4       0.00      0.00      0.00         2

    accuracy                           0.39        57
   macro avg       0.13      0.24      0.17        57
weighted avg       0.20      0.39      0.26        57



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m